In [2]:
# 1) main.py
# Save this exact code to main.py in the project root.
# main.py
try:
model = joblib.load("C:\Users\Mahvish Ayub\Desktop\Machine Learning assingement\Paper\Q3\models\best_model.joblib")
# try to get feature names
if hasattr(model, "feature_names_in_"):
model_feature_names = list(model.feature_names_in_)
elif hasattr(model, "named_steps"):
try:
pre = model.named_steps.get("preprocessor") or model.named_steps.get("prep")
if pre is not None and hasattr(pre, "get_feature_names_out"):
model_feature_names = list(pre.get_feature_names_out())
except Exception:
model_feature_names = None
else:
model_feature_names = None
print("Model loaded successfully from:", "C:\Users\Mahvish Ayub\Desktop\Machine Learning assingement\Paper\Q3\models\best_model.joblib")
print("feature_names_in_:", model_feature_names)
except Exception as e:
print("Failed to load model:", "C:\Users\Mahvish Ayub\Desktop\Machine Learning assingement\Paper\Q3\models\best_model.joblib", "error:", e)
traceback.print_exc()
model = None
model_feature_names = None


# attempt load on startup
try_load_model(MODEL_PATH"C:\Users\Mahvish Ayub\Desktop\Machine Learning assingement\Paper\Q3\models\best_model.joblib")


# Flexible input schema: accept arbitrary dict of features
class InputDict(BaseModel):
__root__: Dict[str, Any]




@app.get("/")
def root():
return {"status": "ok", "model_loaded": model is not None}




@app.get("/metadata")
def metadata():
meta = {"model_loaded": model is not None}
meta["feature_names"] = model_feature_names
meta["model_type"] = type(model).__name__ if model is not None else None
return meta




@app.post("/predict")
def predict(payload: InputDict):
if model is None:
return {"error": "Model not loaded on server. Check MODEL_PATH."}


data_dict = payload.__root__
if not isinstance(data_dict, dict):
return {"error": "Invalid payload - expected a JSON object of features."}


try:
df = pd.DataFrame([data_dict])
except Exception as e:
return {"error": f"Could not convert payload to DataFrame: {e}"}


try:
pred = model.predict(df)
pred_val = pred[0] if len(pred) > 0 else None
except Exception as e:
return {"error": f"model.predict failed: {e}"}


proba = None
try:
if hasattr(model, "predict_proba"):
proba_arr = model.predict_proba(df)
if proba_arr.ndim == 2 and proba_arr.shape[1] == 2:
proba = float(proba_arr[0, 1])
else:
proba = proba_arr[0].tolist()
except Exception:
proba = None


return {"prediction": pred_val, "probability": proba}

IndentationError: expected an indented block after 'try' statement on line 4 (938423666.py, line 5)